We trained a ResNet50 model in the [training notebook](https://www.kaggle.com/ankursingh12/fastai-plant2021-training). In this notebook, we will do the inference part. Are you excited? because I am 😁

Lets import fastai & set the seed for reproducibility.

In [ ]:
from fastai.vision.all import *

seed = 42
set_seed(seed, reproducible=True)

Lets initialize our path variable

In [ ]:
path = Path('../input/plant-pathology-2021-fgvc8')
models_path = Path('../input/plant-models')

models = [models_path/f'resnet50_foldx{i}.pkl' for i in range(1)]
models

Next, we will need to define `get_x` and `get_y`. 

**Explanation:** When we export a fastai model, it saves everything (quite literally). Along with the model, the dataloader and its configurations are also saved. There is only one caveat, we cannot pickle functions. Hence, we will have to define them again. 

In [ ]:
def get_x(x): return str(path/'test_images') + os.path.sep + x['image']
def get_y(y): return y['labels']

## Predictions

In [ ]:
## Data
sample = pd.read_csv(path/'sample_submission.csv')
sample

time to load the model and assign test set.

We are all set to make predictions

In [ ]:
predictions = 0
for m in models:
    learner = load_learner(m, cpu=False).to_fp32()
    test_dl = learner.dls.test_dl(sample)
    preds, _ = learner.tta(dl=test_dl)
    predictions += preds
    
predictions /= len(models)
predictions

These predictions are not usable. We will have to convert them their corresponding labels. Please check the [competition home page](https://www.kaggle.com/c/plant-pathology-2021-fgvc8/overview/evaluation) for details about submission format. 

In [ ]:
vocab = learner.dls.vocab
preds_dec = [vocab[i] for i in predictions.argmax(1)]
preds_dec

It worked as expected! We are not done yet. Lets update our submission file 

### Submission

In [ ]:
sample['labels'] = preds_dec
sample.to_csv('submission.csv',index=False)
sample

WoooWW! Thank you show much for sticking with me till the end. 

I wish you learnt something new. Please consider **upvoting** the notebook. Its will encourage me to write more such kernels. See you around, Bye!